## Data praparation

### build the model

Increasing the complexity of a neural network by adding more hidden layers and increasing the number of nodes (neurons) in each layer can help the network learn more complex patterns from the data. However, it also increases the risk of overfitting, especially if you don't have enough data or if the data isn't complex enough to justify the larger model.

Considerations:

- Regularization: With a larger network, consider implementing regularization techniques such as dropout or L2 regularization to prevent overfitting.
- Batch Normalization: Adding batch normalization after each layer can also help by normalizing the inputs to each layer, speeding up training, and providing some regularization.
- Data Size and Complexity: Ensure that your data is complex enough to justify this network size, and you have sufficient data to train it without overfitting.

In [3]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split


np.random.seed(42)


x1 = np.random.rand(10000) * 2 - 1
x2 = np.random.rand(10000) * 2 - 1

y = np.exp(x1) + 0.1 * x2/4  # Relationship

X = np.column_stack((x1, y))  # Inputs are x1 and y
targets = x2  


X_train, X_test, y_train, y_test = train_test_split(X, targets, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 100)  # Two input features: x1 and y
        self.fc2 = nn.Linear(100, 200)
        self.fc3 = nn.Linear(200, 300)
        self.fc4 = nn.Linear(300, 200)
        self.fc5 = nn.Linear(200, 100)
        self.fc6 = nn.Linear(100, 1)  # Output one value: x2

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.fc6(x)
        return x

model = Net()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


def train_model(num_epochs):
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs.view(-1), y_train_tensor)
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')


train_model(1000)
model.eval()
with torch.no_grad():
    predicted_x2 = model(X_test_tensor).view(-1)
    test_loss = criterion(predicted_x2, y_test_tensor)
    print(f'Test Loss: {test_loss.item()}')





Epoch 0, Loss: 0.337266743183136
Epoch 100, Loss: 0.3427073359489441
Epoch 200, Loss: 0.40452417731285095
Epoch 300, Loss: 0.05604883283376694
Epoch 400, Loss: 0.05851827934384346
Epoch 500, Loss: 0.03983382135629654
Epoch 600, Loss: 0.011629586108028889
Epoch 700, Loss: 0.03202347829937935
Epoch 800, Loss: 0.003999611362814903
Epoch 900, Loss: 0.006604017224162817
Test Loss: 0.01817963644862175


In [4]:
X.shape

(10000, 2)

In [5]:
import plotly.graph_objects as go
import numpy as np

actual_x2 = y_test_tensor.numpy()
predicted_x2 = predicted_x2.numpy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=actual_x2, y=predicted_x2, mode='markers', name='Prediction'))
fig.add_trace(go.Scatter(x=actual_x2, y=actual_x2, mode='lines', name='Ideal Fit', line=dict(color='red')))

fig.update_layout(
    title="Actual x2 vs. Predicted x2",

    xaxis=dict(
        title='"Actual x2",',
        # type='log',
        title_font=dict(size=18),
        tickfont=dict(size=18),
        showgrid=True,
        gridcolor='lightgrey'
    ),

    yaxis=dict(
        title="Predicted x2",
        # type='log',
        title_font=dict(size=18),
        tickfont=dict(size=18),
        showgrid=True,
        gridcolor='lightgrey'
    ),

    legend_title="Legend",
    width=900, height=800,
    title_x=0.5,
    title_y=0.95,
    template='plotly_white'
)

fig.show()



In [2]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import torch.nn.functional as F



np.random.seed(42)


x1 = np.random.rand(100) * 2 - 1
x2 = np.random.rand(100) * 2 - 1
y = np.exp(x1) + 0.1 * x2/4  # Relationship

#X = np.column_stack((x1, y))  # Inputs are x1 and y
X = x1[:,None]
targets = y[:,None] #x2  


X_train, X_test, y_train, y_test = train_test_split(X, targets, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)



class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Adjust the architecture as specified
        self.fc1 = nn.Linear(2, 20)  
        self.fc2 = nn.Linear(20, 1) 
        self.x2 = torch.nn.Parameter(torch.normal(0,1,size=(80,1)))

    def forward(self, x):
        # cat x, x2
        x = torch.cat([x, self.x2], dim=1)
        #print(x.shape)

        x = F.silu(self.fc1(x)) 
        
        x = self.fc2(x)  # No activation on output layer for regression
        return x

# Initialize model, loss function, and optimizer
model = Net()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)




def train_model(num_epochs):
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)  # Ensure dimensionality matches
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

# Train the model for 1000 epochs
train_model(1000)

# Evaluate the model's performance
'''model.eval()
with torch.no_grad():
    predicted_x2 = model(X_test_tensor).view(-1)
    test_loss = criterion(predicted_x2, y_test_tensor)
    print(f'Test Loss: {test_loss.item()}')'''



Epoch 0, Loss: 2.1879734992980957
Epoch 100, Loss: 0.007923679426312447
Epoch 200, Loss: 0.0031976974569261074
Epoch 300, Loss: 0.0011897922959178686
Epoch 400, Loss: 0.0009811494965106249
Epoch 500, Loss: 0.0009201637585647404
Epoch 600, Loss: 0.0008557280525565147
Epoch 700, Loss: 0.0007864569197408855
Epoch 800, Loss: 0.0007122050737962127
Epoch 900, Loss: 0.0006336335791274905


"model.eval()\nwith torch.no_grad():\n    predicted_x2 = model(X_test_tensor).view(-1)\n    test_loss = criterion(predicted_x2, y_test_tensor)\n    print(f'Test Loss: {test_loss.item()}')"

In [ ]:
# plot x2, model.x2
# plot y, model.x2

In [12]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import torch.nn.functional as F


np.random.seed(42)


x1 = np.random.rand(100) * 2 - 1
x2 = np.random.rand(100) * 2 - 1
y = np.exp(x1) + 0.1 * x2/4  # Adjusted relationship

X = np.column_stack((x1, y))
targets = x2[:,None]


X_train, X_test, y_train, y_test = train_test_split(X, targets, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)  
        self.fc2 = nn.Linear(20, 1)  

    def forward(self, x):

        x = F.silu(self.fc1(x)) 
        x = self.fc2(x)  
        return x


model = Net()
criterion = nn.MSELoss()  # Mean squared error loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Adam optimizer


def train_model(num_epochs):
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        #loss = criterion(outputs.view(-1), y_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')


train_model(1000)


model.eval()
with torch.no_grad():
    predicted_x2 = model(X_test_tensor).view(-1)
    test_loss = criterion(predicted_x2, y_test_tensor)
    print(f'Test Loss: {test_loss.item()}')


Epoch 0, Loss: 0.3296200931072235
Epoch 100, Loss: 0.24680082499980927
Epoch 200, Loss: 0.12494490295648575
Epoch 300, Loss: 0.0655791163444519
Epoch 400, Loss: 0.03703384846448898
Epoch 500, Loss: 0.02474101446568966
Epoch 600, Loss: 0.018480975180864334
Epoch 700, Loss: 0.015265196561813354
Epoch 800, Loss: 0.013180973939597607
Epoch 900, Loss: 0.012122664600610733
Test Loss: 0.6996623277664185


/opt/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([20, 1])) that is different to the input size (torch.Size([20])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



In [13]:
import plotly.graph_objects as go
import numpy as np

actual_x2 = y_test_tensor.numpy()
predicted_x2 = predicted_x2.numpy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=actual_x2[:,0], y=predicted_x2, mode='markers', name='Prediction'))
fig.add_trace(go.Scatter(x=actual_x2[:,0], y=actual_x2[:,0], mode='lines', name='Ideal Fit', line=dict(color='red')))

fig.update_layout(
    title="Actual x2 vs. Predicted x2",

    xaxis=dict(
        title='"Actual x2",',
        # type='log',
        title_font=dict(size=18),
        tickfont=dict(size=18),
        showgrid=True,
        gridcolor='lightgrey'
    ),

    yaxis=dict(
        title="Predicted x2",
        # type='log',
        title_font=dict(size=18),
        tickfont=dict(size=18),
        showgrid=True,
        gridcolor='lightgrey'
    ),

    legend_title="Legend",
    width=900, height=800,
    title_x=0.5,
    title_y=0.95,
    template='plotly_white'
)

fig.show()



In [9]:
predicted_x2.shape

(20,)

In [10]:
actual_x2.shape

(20, 1)

In [60]:
import plotly.graph_objects as go
import numpy as np

# Assuming x2_tensor and predicted_x2 are available as numpy arrays or tensors. 
# For demonstration, we'll create mock data since actual computation isn't performed here

np.random.seed(42)
actual_x2 = np.random.normal(0, 1, 1000)  # Mock actual x2
predicted_x2 = actual_x2 + np.random.normal(0, 0.1, 1000)  # Mock predictions with some noise

# Create the figure
fig = go.Figure()

# Add actual x2 trace
fig.add_trace(go.Scatter(x=np.arange(len(actual_x2)), y=actual_x2, mode='markers', name='Actual x2'))

# Add predicted x2 trace
fig.add_trace(go.Scatter(x=np.arange(len(predicted_x2)), y=predicted_x2, mode='markers', name='Predicted x2'))

# Update layout
fig.update_layout(
    title="Comparison of Actual x2 and Predicted x2",
    xaxis_title="Index",
    yaxis_title="Value",
    legend_title="Legend"
)

fig.show()


In [36]:
import plotly.graph_objects as go
import numpy as np



np.random.seed(42)
actual_x2 = np.random.normal(0, 1, 1000)  # Mock actual x2
predicted_x2 = actual_x2 + np.random.normal(0, 0.1, 1000)  # Mock predictions with some noise


fig = go.Figure()
fig.add_trace(go.Scatter(x=actual_x2, y=predicted_x2, mode='markers', name='Prediction'))
fig.add_trace(go.Scatter(x=actual_x2, y=actual_x2, mode='lines', name='Ideal Fit', line=dict(color='red')))

fig.update_layout(
    title="Actual x2 vs. Predicted x2",

    xaxis=dict(
        title='"Actual x2",',
        # type='log',
        title_font=dict(size=18),
        tickfont=dict(size=18),
        showgrid=True,
        gridcolor='lightgrey'
    ),

    yaxis=dict(
        title="Predicted x2",
        # type='log',
        title_font=dict(size=18),
        tickfont=dict(size=18),
        showgrid=True,
        gridcolor='lightgrey'
    ),

    legend_title="Legend",
    width=900, height=800,
    title_x=0.5,
    title_y=0.95,
    template='plotly_white'
)

fig.show()



1. Data points align more closely along the diagonal, it indicates that your model is effectively learning the relationship between the inputs and the target outputs.

2. Overfitting: While it's good news that the model fits the training data well, make sure it isn't overfitting. Overfitting occurs when a model learns the training data too well, including the noise and fluctuations, and performs poorly on unseen data. To check for overfitting, you should evaluate the model on a separate validation set that wasn't used during training.

3. Generalization: Ensuring your model generalizes well to new, unseen data is crucial. This can be assessed through cross-validation techniques or by setting aside a portion of your data as a test set.

4. Complexity and Efficiency: Increasing the number of layers and neurons can make your model more complex and computationally expensive. It's often beneficial to find a balance where the model is complex enough to capture the underlying patterns but not so complex that it becomes inefficient or overfits.

## What if introduce more variables?

In [70]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split


np.random.seed(42)

x1 = np.random.normal(size=10000)
x2 = np.random.normal(size=10000)
x3 = np.random.normal(size=10000)
y = np.exp(x1) + 0.1 * x2**2 + np.log(np.abs(x3) + 1e-10) / 2


X = np.column_stack((x1, y))
targets = np.column_stack((x2, x3))


X_train, X_test, targets_train, targets_test = train_test_split(X, targets, test_size=0.2, random_state=42)


X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
targets_train_tensor = torch.tensor(targets_train, dtype=torch.float32)
targets_test_tensor = torch.tensor(targets_test, dtype=torch.float32)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 100)  # Two input features: x1 and y
        self.fc2 = nn.Linear(100, 200)
        self.fc3 = nn.Linear(200, 300)
        self.fc4 = nn.Linear(300, 200)
        self.fc5 = nn.Linear(200, 100)
        self.fc6 = nn.Linear(100, 2)  # Output two values: x2 and x3

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.fc6(x)
        return x





model = Net()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_model(num_epochs):
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, targets_train_tensor)
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

train_model(1000)

model.eval()
with torch.no_grad():
    predicted_targets = model(X_test_tensor)
    test_loss = criterion(predicted_targets, targets_test_tensor)
    print(f'Test Loss: {test_loss.item()}')


actual_x2 = targets_test_tensor[:, 0].numpy()
predicted_x2 = predicted_targets[:, 0].numpy()
actual_x3 = targets_test_tensor[:, 1].numpy()
predicted_x3 = predicted_targets[:, 1].numpy()



Epoch 0, Loss: 0.9987512826919556
Epoch 100, Loss: 0.9789406657218933
Epoch 200, Loss: 0.9739842414855957
Epoch 300, Loss: 0.9703194499015808
Epoch 400, Loss: 0.9624235033988953
Epoch 500, Loss: 0.9576471447944641
Epoch 600, Loss: 0.9499396681785583
Epoch 700, Loss: 0.9453222751617432
Epoch 800, Loss: 0.9346379041671753
Epoch 900, Loss: 0.9352720975875854
Test Loss: 1.072524905204773


In [71]:
import plotly.graph_objects as go


fig_x2 = go.Figure()
fig_x2.add_trace(go.Scatter(x=actual_x2, y=predicted_x2, mode='markers', name='Predicted vs Actual'))
fig_x2.add_trace(go.Scatter(x=actual_x2, y=actual_x2, mode='lines', name='Ideal Fit', line=dict(color='red')))
fig_x2.update_layout(title='Actual x2 vs Predicted x2', xaxis_title='Actual x2', yaxis_title='Predicted x2', legend_title='Legend')
fig_x2.show()


fig_x3 = go.Figure()
fig_x3.add_trace(go.Scatter(x=actual_x3, y=predicted_x3, mode='markers', name='Predicted vs Actual'))
fig_x3.add_trace(go.Scatter(x=actual_x3, y=actual_x3, mode='lines', name='Ideal Fit', line=dict(color='red')))
fig_x3.update_layout(title='Actual x3 vs Predicted x3', xaxis_title='Actual x3', yaxis_title='Predicted x3', legend_title='Legend')
fig_x3.show()
